<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/Asyncio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Asyncio**

In [5]:
!python --version

Python 3.6.7


In [1]:
!pip install --upgrade pip aiohttp aiofiles

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (18.1)
    100% |████████████████████████████████| 1.2MB 12.9MB/s 
    100% |████████████████████████████████| 256kB 25.1MB/s 
    100% |████████████████████████████████| 317kB 24.7MB/s 
  Running setup.py bdist_wheel for idna-ssl ... - done
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


* 1# Вам необходим некий экземпляр цикла прежде чем вы можете запустить какие бы то ни было сопрограммы, а это именно он и есть. На самом деле, где бы вы его не вызывали, get_event_loop() предоставит вам всякий раз тот же самый экземпляр loop, 
раз уж вы используете только один единственный поток.  
* 2# В приведённом ниже коде loop.create_task(main()) является неким особенным вызовом. Ваша функция сопрограммы не будет исполняться пока вы не сделает его. Мы говорим, что create_task() планирует исполнение вашей сопрограммы в данном цикле. Возвращаемый объект task может применяться для отслеживания текущего состояния данной задачи, например, исполняется ли она ещё или уже выполнена, а помимо этого может также применяться для получения некоторого результирующего значения от вашей завершённой сопрограммы. Вы также можете завершать данную задачу при помощи task.cancel(). [с другой стороны: вы можете заметить в этой строке кода что основным параметром в таком вызове функции для create_task() является coro. Именно это соглашение используется в большей части документации API, которую вы обнаружите, и он ссылается на некую coroutine; т.е., строго говоря, собственно результат вызова некоторой функции async def, а не саму эту функцию.]
* 3# Это два способа запуска данного цикла. Они оба заблокируют данный текущий поток, которым обычно является самый основной поток {main}. Отметим, что run_until_complete() сохранит данный цикл исполняемым до тех пор, пока не завершится заданный coro - но все прочие запланированные задачи в этом цикле также будут исполняться при выполнении данного цикла.
* 4# Типичная манера для большинства программ будет состоять в том, чтобы начаться с loop.run_forever() для самой "главной" части ваше программы {main} , а затем, когда получен некий сигнал процесса, остановить данный цикл, собрать всё ещё приостановленные задачи, а затем воспользоваться loop.run_until_complete() до тех пор, пока эти задачи не выполнятся. именно этот метод служит для выполнения сбора. В более общем плане он также может применяться для сбора множества сопрограмм воедино и ожидания (при помощи await!) пока все собранные задачи не завершатся.
* 5# Как уже описано выше, они применяются для постепенного останова некоторой программы. stop() обычно вызывается как следствие получения некоторого сигнала выключения, а close() обычно является самым последним действием: и оно очистит все очереди и остановит Исполнителя {Executor}. "Остановленный" цикл может быть запущен вновь, а "закрытый" цикл исчезает навсегда.

In [7]:

# quickstart.py
import time
import asyncio

async def main():
    print(f'{time.ctime()} Hello!')
    await asyncio.sleep(1.0)
    print(f'{time.ctime()} Goodbye!')
    loop.stop()

        
loop = asyncio.get_event_loop() #1



loop.create_task(main()) #2 
loop.run_forever() #3
pending = asyncio.Task.all_tasks(loop=loop)
group = asyncio.gather(*pending, return_exceptions=True) #4
loop.run_until_complete(group) #3
loop.close() #5

Mon Jan 21 11:51:18 2019 Hello!
Mon Jan 21 11:51:19 2019 Goodbye!


* (1) blocking() обычно вызывает внутри себя time.sleep(), что привело бы к блокировке вашего основного потока и предотвращало бы исполнение вашего цикла событий. Это означает, что вам не следует делать такую функцию сопрограммой, но что ещё более серьёзно, вы не можете вызывать эту функцию из любого места в своём основном потоке, который является именно тем местом, в котором исполняется основной цикл asyncio. Мы решаем данную проблему запуская эту функцию в некотором исполнителе (executor).

* (2) Безотносительно к данному разделу, но нечто, что следует держать на уме далее в этой книге: отметим, что значение времени сна с блокировкой (0.5 секунды) короче чем значение засыпания без блокировки (1 секунда) в вашей сопрограмме main(). Это делает данный код примера изящным и опрятным. В своём следующем разделе мы объясним что бы произошло если бы функции исполнителя переживали бы свои асинхронные эквиваленты во время последовательности выключения.

* (3) await loop.run_in_executor(None, func) Это самый последний момент в нашем списке существенных функций asyncio, о которых следует знать. Иногда вам требуется запускать вещи в некотором отдельном потоке, или даже в отдельном процессе: данный метод применяется именно для этого. Здесь мы передаём свою блокирующую функцию для запуска в определённом по умолчанию исполнителе. [к сожалению, самым первым параметром run_in_executor() является сам экземпляр Executor, который следует применять и вы обязаны передать None чтобы использовать установленный по умолчанию. Всякий раз, как я применяю это, я ощущаю как мой параметр "исполнитель" плачет, будучи вынужденным именоваться ключевым словом аргумента с определённым по умолчанию значением None.] 
Отметим, что loop.run_in_executor() возвращает некое Future, что означает, что вы можете await {подождать} его при вызове в рамках функции другой сопрограммы.

* (4) Продолжаем отмечать вслед за элементом 2: наш набор задач в pending не содержит некоего элемента для того вызова blocking(), который делается в run_in_executor(). Это будет справедливо для любого вызова, который возвращает некое Future вместо какой- то Task. Имеющаяся документация достаточно хорошо определяет возвращаемые типы, поэтому об этом несложно знать; всего лишь просто запомните, что all_tasks() на самом деле возвращает только Task, и никаких Future.

In [2]:
# требуется перезагрузить ядро после выполнения предыдущего примера
# quickstart_exe.py
import time
import asyncio

async def main():
    print(f'{time.ctime()} Hello!')
    await asyncio.sleep(1.0)
    print(f'{time.ctime()} Goodbye!')
    loop.stop()

def blocking(): #1
    time.sleep(0.5) #2
    print(f"{time.ctime()} Hello from a thread!")

loop = asyncio.get_event_loop()

loop.create_task(main())
loop.run_in_executor(None, blocking) #3

loop.run_forever()

pending = asyncio.Task.all_tasks(loop=loop) #4
group = asyncio.gather(*pending)
loop.run_until_complete(group)
loop.close()

Mon Jan 21 12:10:15 2019 Hello!
Mon Jan 21 12:10:16 2019 Hello from a thread!
Mon Jan 21 12:10:16 2019 Goodbye!


In [2]:
import random
from time import sleep
import asyncio


def task(pid):
    """Synchronous non-deterministic task.
    """
    sleep(random.randint(0, 2) * 0.001)
    print('Task %s done' % pid)


async def task_coro(pid):
    """Coroutine non-deterministic task
    """
    await asyncio.sleep(random.randint(0, 2) * 0.001)
    print('Task %s done' % pid)


def synchronous():
    for i in range(1, 10):
        task(i)


async def asynchronous():
    tasks = [asyncio.ensure_future(task_coro(i)) for i in range(1, 10)]
    await asyncio.wait(tasks)


print('Synchronous:')
synchronous()

ioloop = asyncio.get_event_loop()
print('Asynchronous:')
ioloop.run_until_complete(asynchronous())
ioloop.close()

Synchronous:
Task 1 done
Task 2 done
Task 3 done
Task 4 done
Task 5 done
Task 6 done
Task 7 done
Task 8 done
Task 9 done
Asynchronous:
Task 2 done
Task 4 done
Task 5 done
Task 6 done
Task 9 done
Task 1 done
Task 3 done
Task 7 done
Task 8 done


In [0]:
import aiohttp
import asyncio

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'http://python.org')
        print(html)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [3]:
import time
import urllib.request
import asyncio
import aiohttp

URL = 'https://api.github.com/events'
MAX_CLIENTS = 3


def fetch_sync(pid):
    print('Fetch sync process {} started'.format(pid))
    start = time.time()
    response = urllib.request.urlopen(URL)
    datetime = response.getheader('Date')

    print('Process {}: {}, took: {:.2f} seconds'.format(
        pid, datetime, time.time() - start))

    return datetime


async def fetch_async(pid):
    print('Fetch async process {} started'.format(pid))
    start = time.time()
    response = await aiohttp.request('GET', URL)
    datetime = response.headers.get('Date')

    print('Process {}: {}, took: {:.2f} seconds'.format(
        pid, datetime, time.time() - start))

    response.close()
    return datetime


def synchronous():
    start = time.time()
    for i in range(1, MAX_CLIENTS + 1):
        fetch_sync(i)
    print("Process took: {:.2f} seconds".format(time.time() - start))


async def asynchronous():
    start = time.time()
    tasks = [asyncio.ensure_future(
        fetch_async(i)) for i in range(1, MAX_CLIENTS + 1)]
    await asyncio.wait(tasks)
    print("Process took: {:.2f} seconds".format(time.time() - start))


print('Synchronous:')
synchronous()

print('Asynchronous:')
ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
ioloop.close()

Synchronous:
Fetch sync process 1 started
Process 1: Tue, 22 Jan 2019 10:13:59 GMT, took: 0.31 seconds
Fetch sync process 2 started
Process 2: Tue, 22 Jan 2019 10:13:59 GMT, took: 0.26 seconds
Fetch sync process 3 started


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fc053687a20>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fc053687a20>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fc053678f98>
Task exception was never retrieved
future: <Task finished coro=<fetch_async() done, defined at <ipython-input-3-0f35650d1944>:22> exception=TypeError("object _SessionRequestContextManager can't be used in 'await' expression",)>
Traceback (most recent call last):
  File "<ipython-input-3-0f35650d1944>", line 25, in fetch_async
    response = await aiohttp.request('GET', URL)
TypeError: object _SessionRequestContextManager can't be used in 'await' expression
Task exception was never retrieved
future: <Task finished coro=<fetch_async() done, defined at <ipython-input-3-0f35650d1944>:22> exception=TypeError("object _SessionRequestContextManager can't be used in 'await' expression",)>
Traceback (most recent 

Process 3: Tue, 22 Jan 2019 10:14:00 GMT, took: 0.26 seconds
Process took: 0.82 seconds
Asynchronous:
Fetch async process 1 started
Fetch async process 2 started
Fetch async process 3 started
Process took: 0.01 seconds


In [16]:
# требуется перезагрузить ядро после выполнения предыдущего примера
# quickstart_exe.py
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor ############
import multiprocessing                         ##################
print("RUN", time.ctime())
async def main():
    print(f'{time.ctime()} Hello!')
    await asyncio.sleep(1.0)
    print(f'{time.ctime()} Goodbye!')
    loop.stop()

def blocking(): #1
    time.sleep(2.0) #2
    print(f"{time.ctime()} Hello from a thread!")
pool = ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()) #####
loop = asyncio.get_event_loop()

loop.create_task(main())
loop.run_in_executor(pool, blocking)                 #None, blocking) #3

loop.run_forever()

pending = asyncio.Task.all_tasks(loop=loop) #4
group = asyncio.gather(*pending)
loop.run_until_complete(group)
#loop.close()

RUN Tue Jan 22 13:46:29 2019
Tue Jan 22 13:46:29 2019 Hello!
Tue Jan 22 13:46:30 2019 Goodbye!


[]